# 5. LSTM

## 5.1 데이터 준비

In [1]:
# 데이터 조작
import pandas as pd
import numpy as np

# 시각화
import plotly
import plotly.graph_objects as go

In [2]:
# 데이터 불러오기
df = pd.read_csv('data/2017_2021_data.csv', encoding='euc-kr', usecols=[1, 2, 3])

# 날짜 컬럼을 '날짜' 데이터 타입으로 변경
df['날짜'] = pd.to_datetime(df['날짜'])

# 가격 데이터 타입 '실수형'으로 변경
df['고구마가격'] = df['고구마가격'].astype('float')
df['오이가격'] = df['오이가격'].astype('float')

# 데이터셋 정보 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1226 entries, 0 to 1225
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      1226 non-null   datetime64[ns]
 1   고구마가격   1226 non-null   float64       
 2   오이가격    1226 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 28.9 KB


In [3]:
# 결측치 확인
df.isnull().sum()

날짜       0
고구마가격    0
오이가격     0
dtype: int64

In [4]:
## 데이터 준비
# 고구마
sp_price = df['고구마가격'].values
sp_price = sp_price.reshape((-1,1))

# 오이
ccb_price = df['오이가격'].values
ccb_price = ccb_price.reshape((-1,1))

## 5.2 train/test 데이터셋 분할
- 시계열 데이터이기 때문에 랜덤으로 추출하여 분할하는 sklearn의 train_test_split()을 사용할 수 없음
- 2017~2021년도 중 4년은 학습, 1년을 검증용 데이터로 사용

In [5]:
## train/test 분할
# 공통
split_percent = 0.797 # 977행 = 20년 12월 31일
split = int(split_percent*len(ccb_price))

date_train = df['날짜'][:split]
date_test = df['날짜'][split:]

# 고구마
sp_train = sp_price[:split]
sp_test = sp_price[split:]

# 오이
ccb_train = ccb_price[:split]
ccb_test = ccb_price[split:]

# train, test 데이터셋 데이터 개수 확인
sp_train.shape, sp_test.shape, ccb_train.shape, ccb_test.shape

((977, 1), (249, 1), (977, 1), (249, 1))

In [6]:
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator

In [7]:
## 시간 분할
# lookback 크기만큼 전에 있는 데이터와 비교
# 현재 예제는 일간 기준이므로 7일전 데이터를 확인하여 다음을 예측함
look_back = 7

# 고구마
train_generator_sp = TimeseriesGenerator(sp_train, sp_train, length=look_back, batch_size=20)
test_generator_sp = TimeseriesGenerator(sp_test, sp_test, length=look_back, batch_size=1)

# 오이
train_generator_ccb = TimeseriesGenerator(ccb_train, ccb_train, length=look_back, batch_size=20)
test_generator_ccb = TimeseriesGenerator(ccb_test, ccb_test, length=look_back, batch_size=1)

## 5.3 모델 생성

In [8]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

##### 5.3.1 고구마 모델 생성

In [16]:
# 고구마
model_sp = Sequential()
model_sp.add(LSTM(10,
               activation='relu',
               input_shape=(look_back,1)))
model_sp.add(Dense(1))
model_sp.compile(optimizer='adam', loss='mse')

model_sp.fit(train_generator_sp, epochs=100, verbose=1)

Epoch 1/100
49/49 [==============================] - 1s 5ms/step - loss: 27129818.0000
Epoch 2/100
49/49 [==============================] - 0s 5ms/step - loss: 18892196.0000
Epoch 3/100
49/49 [==============================] - 0s 5ms/step - loss: 10528779.0000
Epoch 4/100
49/49 [==============================] - 0s 5ms/step - loss: 3159405.0000
Epoch 5/100
49/49 [==============================] - 0s 5ms/step - loss: 192359.6094
Epoch 6/100
49/49 [==============================] - 0s 4ms/step - loss: 26670.6816
Epoch 7/100
49/49 [==============================] - 0s 6ms/step - loss: 22262.4082
Epoch 8/100
49/49 [==============================] - 0s 5ms/step - loss: 26735.8457
Epoch 9/100
49/49 [==============================] - 0s 5ms/step - loss: 24212.1367
Epoch 10/100
49/49 [==============================] - 0s 4ms/step - loss: 23569.2617
Epoch 11/100
49/49 [==============================] - 0s 4ms/step - loss: 22363.3203
Epoch 12/100
49/49 [==============================] - 0s 4ms/s

##### 5.3.2 오이 모델 생성

In [10]:
# 오이
model_ccb = Sequential()
model_ccb.add(LSTM(10,
                  activation='relu',
                  input_shape=(look_back,1)))
model_ccb.add(Dense(1))
model_ccb.compile(optimizer='adam', loss='mse')

model_ccb.fit(train_generator_ccb, epochs=100, verbose=1)

Epoch 1/100
49/49 [==============================] - 1s 5ms/step - loss: 2002291.7500
Epoch 2/100
49/49 [==============================] - 0s 5ms/step - loss: 583294.5625
Epoch 3/100
49/49 [==============================] - 0s 5ms/step - loss: 576055.3750
Epoch 4/100
49/49 [==============================] - 0s 5ms/step - loss: 584034.7500
Epoch 5/100
49/49 [==============================] - 0s 6ms/step - loss: 577150.8750
Epoch 6/100
49/49 [==============================] - 0s 5ms/step - loss: 572399.8750
Epoch 7/100
49/49 [==============================] - 0s 6ms/step - loss: 579899.3750
Epoch 8/100
49/49 [==============================] - 0s 4ms/step - loss: 576395.1250
Epoch 9/100
49/49 [==============================] - 0s 5ms/step - loss: 576183.0625
Epoch 10/100
49/49 [==============================] - 0s 4ms/step - loss: 568488.9375
Epoch 11/100
49/49 [==============================] - 0s 4ms/step - loss: 577116.8750
Epoch 12/100
49/49 [==============================] - 0s 5ms/s

## 5.4 모델 예측
##### 5.4.1 모델 예측

In [17]:
# 고구마
prediction_sp = model_sp.predict(test_generator_sp)

242/242 [==============================] - 1s 2ms/step


In [12]:
# 오이
prediction_ccb = model_ccb.predict(test_generator_ccb)

242/242 [==============================] - 1s 2ms/step


##### 5.4.2 예측 결과 배열 변환

In [18]:
## 시각화를 위한 배열 변환
# 고구마
sp_train = sp_train.reshape((-1))
sp_test = sp_test.reshape((-1))
prediction_sp = prediction_sp.reshape((-1))

# 오이
ccb_train = ccb_train.reshape((-1))
ccb_test = ccb_test.reshape((-1))
prediction_ccb = prediction_ccb.reshape((-1))

##### 5.4.3 고구마 가격 예측 시각화

In [26]:
# 고구마 가격 예측 시각
trace1 = go.Scatter(x = date_train,
                    y = sp_train,
                    mode = 'lines',
                    name = 'Data')


trace2 = go.Scatter(x = date_test,
                    y = prediction_sp,
                    mode = 'lines',
                    name = 'Prediction')

trace3 = go.Scatter(x = date_test,
                    y = sp_test,
                    mode='lines',
                    name = 'actual')

layout = go.Layout(title = "고구마 가격 예측",
                   xaxis = {'title' : "Date"},
                   yaxis = {'title' : "Price"})

fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

# plotly.offline.plot(fig, filename='data/sp01.html')
# fig.write_image("data/sp01.pdf")

##### 5.4.4 오이 가격 예측 시각화

In [27]:
# 오이 가격 예측 시각화
trace1 = go.Scatter(x = date_train,
                    y = ccb_train,
                    mode = 'lines',
                    name = 'Data')

trace2 = go.Scatter(x = date_test,
                    y = prediction_ccb,
                    mode = 'lines',
                    name = 'Prediction')

trace3 = go.Scatter(x = date_test,
                    y = ccb_test,
                    mode='lines',
                    name = 'actual')

layout = go.Layout(title = "오이 가격 예측",
                   xaxis = {'title' : "Date"},
                   yaxis = {'title' : "Price"})

fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

# plotly.offline.plot(fig, filename='data/ccb01.html')
# fig.write_image("data/ccb01.pdf")

## 5.5 모델 평가
- MSE, RMSE, MAE, R2 Score
    - R2 Score는 값이 1에 가까울수록, 나머지 평가지표는 값이 작을수록 좋은 모델

##### 5.5.1 평가 전 준비

In [21]:
# lookback으로 줄어든 행 개수 확인
sp_test.shape, prediction_sp.shape, ccb_test.shape, prediction_ccb.shape

((249,), (242,), (249,), (242,))

In [22]:
# 예측 개수에 맞춰 테스트셋 길이 변환 후 score 확인
sp_test = sp_test[0:242]
ccb_test = ccb_test[0:242]

sp_test.shape, prediction_sp.shape, ccb_test.shape, prediction_ccb.shape

((242,), (242,), (242,), (242,))

##### 5.5.2 모델 평가

In [23]:
## 모델 평가
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 평가 함수 정의(한번에 4가지 score를 확인)
def get_evaluate(test_data, predict_data, name):
    mse = mean_squared_error(test_data, predict_data)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test_data, predict_data)
    r2 = r2_score(test_data, predict_data)

    print(f'# {name}의 예측 평가 지표')
    print(f'MSE: {mse:.2f}\nRMSE: {rmse:.2f}\nMAE: {mae:.2f}\nR^2: {r2:.2f}')

In [24]:
# 고구마 모델의 성능 평가
get_evaluate(sp_test, prediction_sp, '고구마')

# 고구마의 예측 평가 지표
MSE: 29020.54
RMSE: 170.35
MAE: 135.26
R^2: 0.92


In [25]:
# 오이 모델의 성능 평가
get_evaluate(ccb_test, prediction_ccb, '오이')

# 오이의 예측 평가 지표
MSE: 793269.91
RMSE: 890.66
MAE: 672.76
R^2: 0.85
